In [2]:
import tensorflow as tf
img_dir = "C:/Users/Kacper/Desktop/github/OneFish-Disseration-Project/Model/img"
# Define image size and number of classes
IMG_SIZE = 224
NUM_CLASSES = 30
batch_size = 32

# Load the dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    img_dir,
    validation_split=0.2,
    subset="training",
    seed =123,
    image_size=(IMG_SIZE,IMG_SIZE),
    batch_size= batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  img_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(IMG_SIZE, IMG_SIZE),
  batch_size=batch_size)

Found 7482 files belonging to 30 classes.
Using 5986 files for training.
Found 7482 files belonging to 30 classes.
Using 1496 files for validation.


In [3]:
# Preprocess the data
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
  tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
])
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))
train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))
val_ds = val_ds.map(lambda x, y: (preprocess_input(x), y))


In [4]:
# Build the model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet",
)
base_model.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)

# Train the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

9406464/9406464 [==============================] - 1s 0us/step


In [5]:
import numpy as np
import cv2
import os
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [6]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


Epoch 1/10


c:\Users\Kacper\.conda\envs\tfgpu\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


188/188 [==============================] - 25s 109ms/step - loss: 2.1067 - sparse_categorical_accuracy: 0.4078 - val_loss: 1.6205 - val_sparse_categorical_accuracy: 0.5201
Epoch 2/10
188/188 [==============================] - 21s 110ms/step - loss: 1.2907 - sparse_categorical_accuracy: 0.6171 - val_loss: 1.3691 - val_sparse_categorical_accuracy: 0.5909
Epoch 3/10
188/188 [==============================] - 21s 110ms/step - loss: 1.1333 - sparse_categorical_accuracy: 0.6580 - val_loss: 1.2864 - val_sparse_categorical_accuracy: 0.6090
Epoch 4/10
188/188 [==============================] - 21s 110ms/step - loss: 1.0059 - sparse_categorical_accuracy: 0.6911 - val_loss: 1.2453 - val_sparse_categorical_accuracy: 0.6270
Epoch 5/10
188/188 [==============================] - 21s 110ms/step - loss: 0.9540 - sparse_categorical_accuracy: 0.7058 - val_loss: 1.2198 - val_sparse_categorical_accuracy: 0.6330
Epoch 6/10
188/188 [==============================] - 21s 110ms/step - loss: 0.8989 - sparse_cat

KeyError: 'accuracy'